In [ ]:
import os
import numpy as np
import mne
import osl
import scipy
import yaml
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
clip_num = 50

In [ ]:
def encode(x, pca, robust, maxabs, robust2, mu=255):
    x = robust.fit_transform(x.T)
    x = pca.fit_transform(x)
    print(np.mean(x, axis=0))
    print(np.std(x, axis=0))
    
    x = clip(x.T).T
    x = robust2.fit_transform(x)
    x = maxabs.fit_transform(x).T
    print('-------------------')
    print(np.mean(x, axis=1))
    print(np.std(x, axis=1))
    
    
    
    x = x.reshape(-1)
    x = np.sign(x)*np.log(1+mu*np.abs(x))/np.log(1+mu)
    
    bins = np.linspace(-1, 1, mu + 1)
    x = np.digitize(x, bins) - 1

    return x

In [ ]:
def decode(x, pca, robust, maxabs, robust2, mu=255, shape=(306, 2857000)):
    x = x / (mu + 1) * 2 - 1
    x = np.sign(x)*((mu+1)**np.abs(x)-1) / mu
    
    x = x.reshape(shape)
    x = maxabs.inverse_transform(x.T)
    x = robust2.inverse_transform(x)
    x = pca.inverse_transform(x)
    x = robust.inverse_transform(x)
    
    return x.T

In [ ]:
def clip(x):
    shape = x.shape
    
    sorted_ = np.sort(x)
    clip_vals = sorted_[:, -clip_num]
    
    for i in range(shape[0]):
        x[i, :] = np.clip(x[i, :], -clip_vals[i], clip_vals[i])

    return x

In [ ]:
outdir = os.path.join('rich_data', 'subj2', 'sess4', 'oslpy_deb')

In [ ]:
path = os.path.join(outdir, 'task_part1_4_tsss_preproc_raw.fif')
raw = mne.io.read_raw_fif(path, preload=True)

In [ ]:
raw_data = raw.get_data(picks='meg')

In [ ]:
data = clip(raw_data)

In [ ]:
pca = PCA(306)
robust_scaler = RobustScaler()
robust_scaler2 = RobustScaler()
maxabs = MaxAbsScaler()

In [ ]:
%matplotlib widget
plt.hist(pca_data.reshape(-1), bins=256)
#plt.xlim(-1, 1)
plt.show()

In [ ]:
plt.hist(raw_data[100], bins=256)
#plt.xlim(-1, 1)
plt.show()

In [ ]:
encoded = encode(raw_data, pca, robust_scaler, maxabs, robust_scaler2)

In [ ]:
%matplotlib widget
plt.plot(encoded[50000:55000])

In [ ]:
encoded = encoded.reshape((306, 2857000))

In [ ]:
%matplotlib widget
plt.hist(encoded[1], bins=256)
plt.show()

In [ ]:
%matplotlib widget
plt.hist(encoded[-1], bins=256)
plt.show()

In [ ]:
decoded = decode(encoded.reshape(-1), pca, robust_scaler, maxabs, robust_scaler2)

In [ ]:
plt.hist(decoded, bins=256)
plt.show()

In [ ]:
errors = []
for i in range(306):
    data_mean = np.mean(np.abs(raw_data[i]))
    diff = np.mean(np.abs(raw_data[i] - decoded[i]))
    
    errors.append(diff/data_mean)

In [ ]:
%matplotlib widget
plt.hist(errors, bins=306)
plt.show()

In [ ]:
np.mean(np.abs(data - decoded))

In [ ]:
np.mean(np.abs(data))

In [ ]:
diff = data - decoded

In [ ]:
%matplotlib widget
plt.plot(data[10, 50000:52000], 'r', decoded[10, 50000:52000], 'b', linewidth=0.2)

In [ ]:
%matplotlib widget
plt.hist(diff.reshape(-1), bins=10000)
plt.xlim((-2e-12, 2e-12))

In [ ]:
errors = np.arange(256) / 256

In [ ]:
np.mean(errors**2)

In [ ]:
path = os.path.join('..', 'results', 'cichy_epoched', 'subj1', 'cont_quantized', 'wavenetfulltest', 'preds.npy')
preds = np.load(path)

path = os.path.join('..', 'results', 'cichy_epoched', 'subj1', 'cont_quantized', 'wavenetfulltest', 'targets.npy')
targets = np.load(path)

In [ ]:
preds.shape

In [ ]:
preds = preds.transpose(1, 0, 2).reshape(preds.shape[1], -1)
targets = targets.transpose(1, 0, 2).reshape(targets.shape[1], -1)

In [ ]:
np.mean((targets[:, :, 1:].reshape(-1) - targets[:, :, :-1].reshape(-1))**2)

In [ ]:
np.mean((preds[:, :, 1:].reshape(-1) - targets[:, :, :-1].reshape(-1))**2)

In [ ]:
mse

In [ ]:
%matplotlib widget
plt.plot(preds[0, 0, 1:60], linewidth=0.5)
plt.plot(targets[0, 0, :60], linewidth=0.5)

In [1]:
np.min(np.abs(preds.reshape(-1)))

NameError: name 'np' is not defined

In [ ]:
sum(np.abs(targets.reshape(-1))<0.075)/np.prod(targets.shape)

In [ ]:
%matplotlib widget
plt.scatter(np.arange(len(np.unique(preds))), np.unique(preds), s=1)
#plt.plot(np.unique(targets))

In [ ]:
%matplotlib widget
plt.hist(targets.reshape(-1), bins=1000)
plt.show()

In [ ]:
path = os.path.join('..', 'results', 'cichy_epoched', 'subj1', 'cont_quantized', 'dumb_conv', 'inputs.npy')
inputs = np.load(path)

path = os.path.join('..', 'results', 'cichy_epoched', 'subj1', 'cont_quantized', 'dumb_conv', 'targets_full.npy')
targets = np.load(path)

In [ ]:
inputs = inputs.transpose(1, 0, 2).reshape(inputs.shape[1], -1)
targets = targets.transpose(1, 0, 2).reshape(targets.shape[1], -1)

In [ ]:
%matplotlib widget
plt.plot(inputs[0, 1:10001], linewidth=0.5)
plt.plot(targets[0, :10000], linewidth=0.5)

In [ ]:
%matplotlib widget
plt.scatter(np.arange(len(np.unique(targets))), np.unique(targets), s=1)

In [ ]:
%matplotlib widget
plt.hist(inputs.reshape(-1), bins=400)
plt.show()